Import libraries

In [1]:
import pandas as pd
import numpy as np # library to handle data in a vectorized manner
 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import geopy
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans
!pip install folium
import folium # map rendering library

print('Libraries imported.')


Libraries imported.


International visitors London File csv

visitors-london-country-year

In [2]:
boroughs=pd.read_csv("/home/sedna/Downloads/PROJECT-COURSERA-CAPSTONE/DATA/Boroughs_london.csv",index_col=0)

Next, we are going to start utilizing the Foursquare API to explore the boroughs and segment them.

Define Foursquare Credentials and Version

In [3]:
CLIENT_ID = 'X3INNHZFS0RE4YAMNP5CCG4UVUP4S5K1S4NZIIXKHN2UNXJW' # your Foursquare ID
CLIENT_SECRET = '20EOF0IP0MALLD0RO0MM1QOCLEHXC35L4U3JHMETTEF24YJR' # your Foursquare Secret

VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: X3INNHZFS0RE4YAMNP5CCG4UVUP4S5K1S4NZIIXKHN2UNXJW
CLIENT_SECRET:20EOF0IP0MALLD0RO0MM1QOCLEHXC35L4U3JHMETTEF24YJR


Explore boroughs in London.
Let's create a function to repeat the same process to all the boroughs in London

In [4]:

London_boroughs_venues=pd.read_csv("/home/sedna/Downloads/PROJECT-COURSERA-CAPSTONE/DATA/london_boroughs_venues.csv", index_col=0)


Let's check how many venues were returned for each borough

In [5]:
London_boroughs_venues.groupby('Borough').count()

,Latitude,Longitude,Venue,Venue Latitude,Venue Longitude,Venue category,Venue Category
Borough,,,,,,,
Barking and Dagenham,7,7,7,7,7,7,7
Barnet,3,3,3,3,3,3,3
Bexley,28,28,28,28,28,28,28
Brent,71,71,71,71,71,71,71
Bromley,42,42,42,42,42,42,42
Camden,100,100,100,100,100,100,100
City of London,100,100,100,100,100,100,100
Croydon,40,40,40,40,40,40,40
Ealing,72,72,72,72,72,72,72


Let's find out how many unique categories can be curated from all the returned venues

In [6]:
print('There are {} uniques categories.'.format(len(London_boroughs_venues['Venue category'].unique())))

There are 226 uniques categories.


In [7]:
print(London_boroughs_venues['Venue category'].unique())

['Steakhouse' 'Bookstore' 'Vietnamese Restaurant' 'Hotel' 'Cocktail Bar'
 'Roof Deck' 'Italian Restaurant' 'Lounge' 'Butcher' 'French Restaurant'
 'Seafood Restaurant' 'Art Gallery' 'Coffee Shop' 'Yoga Studio' 'Plaza'
 'Grocery Store' 'Modern European Restaurant' 'Business Service'
 'Gym / Fitness Center' 'New American Restaurant' 'History Museum'
 'Indian Restaurant' 'Scandinavian Restaurant' 'Café' 'Park' 'Wine Bar'
 'Mexican Restaurant' 'Falafel Restaurant' 'Asian Restaurant'
 'Event Space' 'Historic Site' 'Restaurant' 'Udon Restaurant'
 'Pedestrian Plaza' 'English Restaurant' 'Sushi Restaurant' 'Concert Hall'
 'Clothing Store' 'Peruvian Restaurant' 'Theater' 'Garden'
 'Performing Arts Venue' 'Bakery' 'Dessert Shop' 'Botanical Garden'
 'Indie Movie Theater' 'Scenic Lookout' 'Burrito Place' 'Shopping Mall'
 'Pub' 'Boxing Gym' 'Juice Bar' 'Food Truck' 'Portuguese Restaurant'
 'Cycle Studio' 'Japanese Restaurant' 'Golf Course' 'Martial Arts Dojo'
 'Bus Station' 'Pool' 'Supermarket' 'Bu

Analize each borough

In [8]:
# one hot encoding
London_boroughs_onehot = pd.get_dummies(London_boroughs_venues[['Venue category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
London_boroughs_onehot['Borough'] = London_boroughs_venues['Borough'] 

# move neighborhood column to the first column
fixed_columns = [London_boroughs_onehot.columns[-1]] + list(London_boroughs_onehot.columns[:-1])
London_boroughs_onehot = London_boroughs_onehot[fixed_columns]

London_boroughs_onehot.head()


,Borough,African Restaurant,Airport,Airport Lounge,Airport Service,American Restaurant,Antique Shop,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Australian Restaurant,BBQ Joint,Bakery,Bank,Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Bistro,Boarding House,Bookstore,Botanical Garden,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Buffet,Burger Joint,Burrito Place,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Chaat Place,Champagne Bar,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Donut Shop,Dumpling Restaurant,Electronics Store,English Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Film Studio,Fish & Chips Shop,Fish Market,Flea Market,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,German Restaurant,Gift Shop,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hardware Store,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Insurance Office,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kebab Restaurant,Kitchen Supply Store,Korean Restaurant,Latin American Restaurant,Lebanese Restaurant,Light Rail Station,Lighting Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Modern European Restaurant,Movie Theater,Multiplex,Museum,Music Store,Music Venue,New American Restaurant,Nightclub,Noodle House,Okonomiyaki Restaurant,Optical Shop,Organic Grocery,Outdoor Sculpture,Outdoor Supply Store,Outlet Mall,Outlet Store,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pharmacy,Piano Bar,Pizza Place,Platform,Playground,Plaza,Polish Restaurant,Pool,Pool Hall,Portuguese Restaurant,Pub,Public Art,Rafting,Ramen Restaurant,Record Shop,Recording Studio,Restaurant,Road,Roof Deck,Salad Place,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Smoothie Shop,South American Restaurant,Southern / Soul Food Restaurant,Souvenir Shop,Souvlaki Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Street Art,Street Food Gathering,Supermarket,Sushi Restaurant,Szechuan Restaurant,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Thrift / Vintage Store,Tiki Bar,Toy / Game Store,Trail,Train Station,Tunnel,Turkish Restaurant,Udon Restaurant,Used Bookstore,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Winery,Women's Store,Yoga Studio
0,City of London,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,City of London,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

Examine the dataframe size

In [9]:
London_boroughs_onehot.shape

(1611, 227)

Next, let's group rows by borough and by taking the mean of the frequency of occurrence of each category

In [10]:
London_boroughs_grouped = London_boroughs_onehot.groupby('Borough').mean().reset_index()
London_boroughs_grouped


,Borough,African Restaurant,Airport,Airport Lounge,Airport Service,American Restaurant,Antique Shop,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Australian Restaurant,BBQ Joint,Bakery,Bank,Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Bistro,Boarding House,Bookstore,Botanical Garden,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Buffet,Burger Joint,Burrito Place,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Chaat Place,Champagne Bar,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Donut Shop,Dumpling Restaurant,Electronics Store,English Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Film Studio,Fish & Chips Shop,Fish Market,Flea Market,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,German Restaurant,Gift Shop,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hardware Store,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Insurance Office,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kebab Restaurant,Kitchen Supply Store,Korean Restaurant,Latin American Restaurant,Lebanese Restaurant,Light Rail Station,Lighting Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Modern European Restaurant,Movie Theater,Multiplex,Museum,Music Store,Music Venue,New American Restaurant,Nightclub,Noodle House,Okonomiyaki Restaurant,Optical Shop,Organic Grocery,Outdoor Sculpture,Outdoor Supply Store,Outlet Mall,Outlet Store,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pharmacy,Piano Bar,Pizza Place,Platform,Playground,Plaza,Polish Restaurant,Pool,Pool Hall,Portuguese Restaurant,Pub,Public Art,Rafting,Ramen Restaurant,Record Shop,Recording Studio,Restaurant,Road,Roof Deck,Salad Place,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Smoothie Shop,South American Restaurant,Southern / Soul Food Restaurant,Souvenir Shop,Souvlaki Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Street Art,Street Food Gathering,Supermarket,Sushi Restaurant,Szechuan Restaurant,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Thrift / Vintage Store,Tiki Bar,Toy / Game Store,Trail,Train Station,Tunnel,Turkish Restaurant,Udon Restaurant,Used Bookstore,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Winery,Women's Store,Yoga Studio
0,Barking and Dagenham,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.142857,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.00,0.00,0.000000,0.000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.00,0.00000

Let's confirm new size

In [11]:
London_boroughs_grouped.shape

(33, 227)

Let's print each borough with the 10 most common venues

In [12]:
num_top_venues = 10

for hood in London_boroughs_grouped['Borough']:
    print("----"+hood+"----")
    temp = London_boroughs_grouped[London_boroughs_grouped['Borough'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Barking and Dagenham----
                  venue  freq
0                  Park  0.14
1           Bus Station  0.14
2                  Pool  0.14
3           Supermarket  0.14
4           Golf Course  0.14
5     Martial Arts Dojo  0.14
6  Gym / Fitness Center  0.14
7    African Restaurant  0.00
8          Optical Shop  0.00
9       Organic Grocery  0.00


----Barnet----
                   venue  freq
0                   Café  0.67
1               Bus Stop  0.33
2     African Restaurant  0.00
3  Performing Arts Venue  0.00
4           Optical Shop  0.00
5        Organic Grocery  0.00
6      Outdoor Sculpture  0.00
7   Outdoor Supply Store  0.00
8            Outlet Mall  0.00
9           Outlet Store  0.00


----Bexley----
                  venue  freq
0                   Pub  0.11
1           Coffee Shop  0.11
2        Clothing Store  0.11
3           Supermarket  0.07
4  Fast Food Restaurant  0.07
5    Italian Restaurant  0.07
6       Warehouse Store  0.04
7             Multiplex  0

Let's put that into a pandas dataframe. First, let's write a function to sort the venues in descending order.

In [13]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each borough.

In [14]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Borough']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
boroughs_venues_sorted = pd.DataFrame(columns=columns)
boroughs_venues_sorted['Borough'] = London_boroughs_grouped['Borough']

for ind in np.arange(London_boroughs_grouped.shape[0]):
    boroughs_venues_sorted.iloc[ind, 1:] = return_most_common_venues(London_boroughs_grouped.iloc[ind, :], num_top_venues)

boroughs_venues_sorted.head()


,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Barking and Dagenham,Pool,Bus Station,Supermarket,Martial Arts Dojo,Park,Gym / Fitness Center,Golf Course,Farmers Market,Food Truck,Food Court
1,Barnet,Café,Bus Stop,Yoga Studio,Furniture / Home Store,Fried Chicken Joint,French Restaurant,Food Truck,Food Court,Flea Market,Fish Market
2,Bexley,Coffee Shop,Clothing Store,Pub,Supermarket,Italian Restaurant,Fast Food Restaurant,Sandwich Place,Furniture / Home Store,Chinese Restaurant,Optical Shop
3,Brent,Hotel,Coffee Shop,Clothing Store,Bar,Sporting Goods Shop,Sandwich Place,Indian Restaurant,Grocery Store,Italian Restaurant,American Restaurant
4,Bromley,Clothing Store,Coffee Shop,Bar,Pizza Place,Burger Joint,Gym / Fitness Center,Chocolate Shop,Bookstore,Sushi Restaurant,Supermarket


In [15]:
London_boroughs_grouped_clustering = London_boroughs_grouped.drop('Borough', 1)


SSD=norm2(samples,nearest_clusters_centre) Elbow curve to determine the optimal number of clusters



In [16]:
SSD = []
K = range(1,20)
for k in K:
    km = KMeans(n_clusters=k)
    km = km.fit(London_boroughs_grouped_clustering)
    SSD.append(km.inertia_)

ELBOW CURVE FOR KMEANS

In [17]:
import matplotlib.pyplot as plt
def elbow(norm):
    plt.plot(K,norm, 'ro-')
    plt.xlabel('k')
    plt.ylabel('L2-norm')
    plt.title('Elbow Method For Optimal k')
    return plt.show()
elbow_curve=elbow(SSD)
elbow_curve

<Figure size 640x480 with 1 Axes>

In [18]:
np.min(np.array(SSD))


0.19175777992014736

Cluster Boroughs. Run k-means to cluster the boroughs into 10 clusters.



In [19]:
boroughs_venues_sorted.shape

(33, 11)

In [20]:
# set number of clusters
kclusters = 10


# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(London_boroughs_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_
kmeans_labels=kmeans.labels_

In [21]:
from sklearn import metrics
from sklearn.metrics import pairwise_distances
s_score=metrics.silhouette_score(London_boroughs_grouped_clustering,kmeans_labels, metric='euclidean')
s_score=pd.Series(s_score,index=[kclusters])

In [22]:
pp=pd.DataFrame(kmeans.labels_,columns=['Cluster Labels'])
kclusters_b10s=pp['Cluster Labels'].value_counts().sort_index()
kclusters_b10s=kclusters_b10s.append(s_score)
kclusters_b10s

0     11.00000
1      2.00000
2      1.00000
3      1.00000
4      1.00000
5      1.00000
6     13.00000
7      1.00000
8      1.00000
9      1.00000
10     0.07568
dtype: float64

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each borough.

In [23]:
# add clustering labels
boroughs_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
# merge London_borough_grouped with to add latitude/longitude for each borough
London_boroughs_merged =pd.merge(boroughs,boroughs_venues_sorted, on='Borough').sort_values('Cluster Labels')

London_boroughs_merged.head() # check the last columns!

,Code,Borough,Inner/Outer,GLA_P2017,GLA_H2017,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,E09000001,City of London,Inner London,8800,5326,51.5155,-0.0922,0,Coffee Shop,Italian Restaurant,Seafood Restaurant,French Restaurant,Café,Wine Bar,Restaurant,Vietnamese Restaurant,Hotel,Gym / Fitness Center
27,E09000028,Southwark,Inner London,314300,134254,51.5035,-0.0804,0,Coffee Shop,Pub,Restaurant,Bar,Scenic Lookout,French Restaurant,Thai Restaurant,Cocktail Bar,Pizza Place,English Restaurant
26,E09000027,Richmond upon Thames,Outer London,197300,85108,51.4479,-0.3260,0,Pub,Coffee Shop,Italian Restaurant,Indian Restaurant,Steakhouse,Café,Grocery Store,Bus Stop,Gym,Bookstore
21,E09000022,Lambeth,Inner London,328900,144400,51.4607,-0.1163,0,Caribbean Restaurant,Market,Pub,Mexican Restaurant,Gym / Fitness Center,Pizza Place,Beer Bar,Tapas Restaurant,Coffee Shop,Burger Joint
19,E09000020,Kensington and Chelsea,Inner London,159000,80200,51.5020,-0.1947,0,Clothing Store,Café,Hotel,Restaurant,Bakery,Pub,French Restaurant,Modern European Restaurant,Juice Bar,Italian Restaurant


In [24]:
address = 'London,UK'

geolocator = Nominatim(user_agent="to_explorer",timeout=50)
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude


Finally, let's visualize the resulting clusters

In [25]:
# create map
map_boroughs_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**3 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(London_boroughs_merged['Latitude'], London_boroughs_merged['Longitude'],
                                  London_boroughs_merged['Borough'], London_boroughs_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.1).add_to(map_boroughs_clusters)
       
map_boroughs_clusters


In [26]:
map_boroughs_clusters.save("/home/sedna/Downloads/PROJECT-COURSERA-CAPSTONE/RESULTS/map_London_boroughs_clusters_category.html")

Examine Clusters. Now, I can examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, I will assign a name to each cluster.

In [27]:
    print ('CLuster 1: ')

 

CLuster 1: 


In [28]:
 London_boroughs_merged.loc[London_boroughs_merged['Cluster Labels']==0,:]
    
 

,Code,Borough,Inner/Outer,GLA_P2017,GLA_H2017,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,E09000001,City of London,Inner London,8800,5326,51.5155,-0.0922,0,Coffee Shop,Italian Restaurant,Seafood Restaurant,French Restaurant,Café,Wine Bar,Restaurant,Vietnamese Restaurant,Hotel,Gym / Fitness Center
27,E09000028,Southwark,Inner London,314300,134254,51.5035,-0.0804,0,Coffee Shop,Pub,Restaurant,Bar,Scenic Lookout,French Restaurant,Thai Restaurant,Cocktail Bar,Pizza Place,English Restaurant
26,E09000027,Richmond upon Thames,Outer London,197300,85108,51.4479,-0.3260,0,Pub,Coffee Shop,Italian Restaurant,Indian Restaurant,Steakhouse,Café,Grocery Store,Bus Stop,Gym,Bookstore
21,E09000022,Lambeth,Inner London,328900,144400,51.4607,-0.1163,0,Caribbean Restaurant,Market,Pub,Mexican Restaurant,Gym / Fitness Center,Pizza Place,Beer Bar,Tapas Restaurant,Coffee Shop,Burger Joint
19,E09000020,Kensington and Chelsea,Inner London,159000,80200,51.5020,-0.1947,0,Clothing Store,Café,Hotel,Restaurant,Bakery,Pub,French Restaurant,Modern European Restaurant,Juice Bar,Italian Restaurant
18,E09000019,Islington,Inner London,231200,105038,51.5416,-0.1022,0,Pub,Bakery,Mediterranean Restaurant,French Restaurant,Cocktail Bar,Ice Cream Shop,Park,Boutique,Turkish Restaurant,Furniture / Home Store
12,E09000013,Hammersmith and Fulham,Inner London,185300,83552,51.4927,-0.2339,0,Pub,Café,Hotel,Indian Restaurant,Coffee Shop,Italian Restaurant,Gastropub,Chinese Restaurant,Bus Stop,Clothing Store
11,E09000012,Hackney,Inner London,274300,115417,51.5450,-0.0553,0,Pub,Bakery,Coffee Shop,Café,Cocktail Bar,Grocery Store,Middle Eastern Restaurant,Hotel,Clothing Store,Brewery
32,E09000033,Westminster,Inner London,242100,118975,51.4973,-0.1372,0,Coffee Shop,Hotel,Sandwich Place,Theater,Gym / Fitness Center,Italian Restaurant,Sushi Restaurant,Pub,Sporting Goods Shop,Hotel Bar
7,E09000008,Croydon,Outer London,386500,159010,51.3714,-0.0977,0,Pub,Coffee Shop,Asian Restaurant,Gym / Fitness Center,Hotel,Bus Stop,Bookstore,Malay Restaurant,Breakfast Spot,Brewery


In [29]:
 print ('CLuster 2: ')

CLuster 2: 


In [30]:
London_boroughs_merged.loc[London_boroughs_merged['Cluster Labels']==1,:]

,Code,Borough,Inner/Outer,GLA_P2017,GLA_H2017,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
13,E09000014,Haringey,Inner London,278000,115608,51.6000,-0.1119,1,Fast Food Restaurant,Pub,Portuguese Restaurant,Bar,Hotel Bar,Italian Restaurant,Gym / Fitness Center,Grocery Store,Mediterranean Restaurant,Movie Theater
23,E09000024,Merton,Outer London,208100,84201,51.4014,-0.1958,1,Park,Café,Supermarket,Fast Food Restaurant,Sandwich Place,Garden Center,Grocery Store,Bar,Bakery,Burger Joint


In [31]:
print ('Cluster 3: ')

Cluster 3: 


In [32]:
London_boroughs_merged.loc[London_boroughs_merged['Cluster Labels']==2,:]

,Code,Borough,Inner/Outer,GLA_P2017,GLA_H2017,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,E09000003,Barnet,Outer London,389600,151423,51.6252,-0.1517,2,Café,Bus Stop,Yoga Studio,Furniture / Home Store,Fried Chicken Joint,French Restaurant,Food Truck,Food Court,Flea Market,Fish Market


In [33]:
print('Cluster 4: ')

Cluster 4: 


In [34]:
London_boroughs_merged.loc[London_boroughs_merged['Cluster Labels']==3,:]

,Code,Borough,Inner/Outer,GLA_P2017,GLA_H2017,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,E09000015,Harrow,Outer London,252300,92557,51.5898,-0.3346,3,Indie Movie Theater,Convenience Store,Supermarket,Coffee Shop,Platform,Indian Restaurant,Food Truck,Food Court,French Restaurant,Falafel Restaurant


In [35]:
print('Cluster 5:')

Cluster 5:


In [36]:
London_boroughs_merged.loc[London_boroughs_merged['Cluster Labels']==4,:]

,Code,Borough,Inner/Outer,GLA_P2017,GLA_H2017,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
17,E09000018,Hounslow,Outer London,274200,105887,51.4746,-0.368,4,Park,Pizza Place,Café,Bed & Breakfast,Falafel Restaurant,French Restaurant,Food Truck,Food Court,Flea Market,Fish Market


In [37]:
print('Cluster 6: ')

Cluster 6: 


In [38]:
London_boroughs_merged.loc[London_boroughs_merged['Cluster Labels']==5,:]

,Code,Borough,Inner/Outer,GLA_P2017,GLA_H2017,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
24,E09000025,Newham,Inner London,342900,119172,51.5077,0.0469,5,Hotel,Light Rail Station,Sandwich Place,Coffee Shop,Pharmacy,Rafting,Airport Service,Airport Lounge,Airport,Chinese Restaurant


In [39]:
print('Cluster 7: ')

Cluster 7: 


In [40]:
London_boroughs_merged.loc[London_boroughs_merged['Cluster Labels']==6,:]

,Code,Borough,Inner/Outer,GLA_P2017,GLA_H2017,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
9,E09000010,Enfield,Outer London,333000,130328,51.6538,-0.0799,6,Café,Coffee Shop,Clothing Store,Optical Shop,Mobile Phone Shop,Sandwich Place,Gift Shop,Department Store,Shopping Mall,Pharmacy
28,E09000029,Sutton,Outer London,202600,85243,51.3618,-0.1945,6,Pub,Clothing Store,Coffee Shop,Bar,Pizza Place,Café,Women's Store,Italian Restaurant,Sandwich Place,Department Store
25,E09000026,Redbridge,Outer London,304200,110708,51.5590,0.0741,6,Hotel,Fast Food Restaurant,Clothing Store,Grocery Store,Burger Joint,Sandwich Place,Coffee Shop,Bakery,Supermarket,Portuguese Restaurant
22,E09000023,Lewisham,Inner London,303400,131076,51.4452,-0.0209,6,Supermarket,Grocery Store,Coffee Shop,Pub,Train Station,Italian Restaurant,Platform,Cocktail Bar,Dessert Shop,Greek Restaurant
20,E09000021,Kingston upon Thames,Outer London,175400,69849,51.4085,-0.3064,6,Coffee Shop,Café,Clothing Store,Italian Restaurant,Pub,Department Store,Sushi Restaurant,Burger Joint,Ice Cream Shop,Bookstore
8,E09000009,Ealing,Outer London,351600,132663,51.5130,-0.3089,6,Coffee Shop,Clothing Store,Pub,Italian Restaurant,Park,Bakery,Hotel,Bar,Pizza Place,Bus Stop
4,E09000005,Brent,Outer London,332100,121048,51.5588,-0.2817,6,Hotel,Coffee Shop,Clothing Store,Bar,Sporting Goods Shop,Sandwich Place,Indian Restaurant,Grocery Store,Italian Restaurant,American Restaurant
5,E09000006,Bromley,Outer London,327900,140602,51.4039,0.0198,6,Clothing Store,Coffee Shop,Bar,Pizza Place,Burger Joint,Gym / Fitness Center,Chocolate Shop,Bookstore,Sushi Restaurant,Supermarket
31,E09000032,Wandsworth,Inner London,321000,138149,51.4567,-0.1910,6,Pub,Coffee Shop,Clothing Store,Indian Restaurant,Pizza Place,Women's Store,Breakfast Spot,Asian Restaurant,Gym / Fitness Center,Supermarket
15,E09000016,Havering,Outer London,254300,104098,51.5812,0.1837,6,Clothing Store,Hotel,Café,Coffee Shop,Shopping Mall,Bakery,Pub,Bookstore,Fast Food Restaurant,Furniture / Home Store


In [41]:
print('Cluster 8: ')

Cluster 8: 


In [42]:
London_boroughs_merged.loc[London_boroughs_merged['Cluster Labels']==7,:]

,Code,Borough,Inner/Outer,GLA_P2017,GLA_H2017,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,E09000002,Barking and Dagenham,Outer London,209000,78188,51.5607,0.1557,7,Pool,Bus Station,Supermarket,Martial Arts Dojo,Park,Gym / Fitness Center,Golf Course,Farmers Market,Food Truck,Food Court


In [43]:
print('Cluster 9: ')

Cluster 9: 


In [44]:
London_boroughs_merged.loc[London_boroughs_merged['Cluster Labels']==8,:]

,Code,Borough,Inner/Outer,GLA_P2017,GLA_H2017,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
30,E09000031,Waltham Forest,Outer London,276200,105981,51.5908,-0.0134,8,Pub,Coffee Shop,Tea Room,Field,Pool,Pizza Place,Beer Store,Concert Hall,Vegetarian / Vegan Restaurant,Art Gallery


In [45]:
print('Cluster 10: ')

Cluster 10: 


In [46]:
London_boroughs_merged.loc[London_boroughs_merged['Cluster Labels']==9,:]

,Code,Borough,Inner/Outer,GLA_P2017,GLA_H2017,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
29,E09000030,Tower Hamlets,Inner London,304000,123720,51.5099,-0.0059,9,Coffee Shop,Italian Restaurant,Gym / Fitness Center,Hotel,Café,Sandwich Place,Grocery Store,Chinese Restaurant,Steakhouse,Convenience Store
